# Comments to the following code
> - #### Parts 1 and 2 can be run separately
> - #### Raw data: Excel sheet can be obtained from https://doi.org/10.1021/acs.est.8b01452
> - #### Terms `exchange` and `input activity` are used interchangeably

# ------------------------------------------ Part 1 ------------------------------------------

In [ ]:
import pandas as pd
import numpy as np
from copy import copy
import os
import re
import brightway2 as bw

# Still TODO
> - ### First exchange should be in 1 unit - probably not the case for us -> check that

# Define constants

In [ ]:
# Database name
CONSUMPTION_DB_NAME = 'CH consumption 1.0'
# Number of relevant columns in the raw file (df_raw) to extract info about activity
N_ACT_RELEVANT = 11
# Index of the column where activities start
FIRST_ACT_IND = 7
# Number of columns that contain info about one activity
N_COLUMNS_INPUT_ACTIVITY = 5

# Column names for exchanges needed by brightway
EXC_COLUMNS_DICT = {
        'name': 'A', 
        'reference product': 'B', 
        'location': 'C', 
        'amount': 'D', 
        'unit': 'E', 
        'database': 'F', 
        'type': 'G', 
        'categories': 'H',
        'comment': 'I',
    }

# Conversion from type in databases to type that should be in excel file to import a new database
ACTIVITY_TYPE_DICT = {
    'process': 'technosphere',
    'emission': 'biosphere',
}

# Convert data to brightway database format -> all functions

In [ ]:
# Add missing On columns
def complete_columns(df):
    
    column_names = list(df.columns)
    indices = [i for i,el in enumerate(column_names)  if 'Activity' in el]
    column_names_complete = copy(column_names)

    n_el_added = 0
    for ind in indices:
        if 'On' not in column_names[ind-1]:
            act_name = column_names[ind]
            act_number = act_name[act_name.find(' ')+1:]
            column_names_complete.insert(ind+n_el_added, 'On ' + act_number)
            n_el_added += 1
        
    df.columns = column_names_complete[:len(column_names)]
    
    return df

In [ ]:
def create_df_bw(db_name, n_cutoff_cols = len(EXC_COLUMNS_DICT)+3):
    '''
    Create dataframe for a new database in the Brightway format and add the necessary meta information
    '''
    df = pd.DataFrame([['cutoff', n_cutoff_cols], ['database', db_name]], columns=list('AB'))
    df = df.append(pd.Series(), ignore_index=True)
    return df

In [ ]:
def compute_act_unit(df):
    '''
    Depending on whether `Quantity code` is present for a specific activity, 
    set unit to the unit of the first input activity or CHF.
    '''
    if 'Quantity code' in df.keys():
        return df['DB Act 1'].split('(')[1].split(',')[0]
    else:
        return 'CHF'

In [ ]:
def append_activity(df, df_ind):
    '''
    Append activity from row df_ind to the dataframe df in the brightway format
    '''
    # Append empty row
    df = df.append(pd.Series(), ignore_index=True)
    
    # Extract activity information
    act_name = df_ind['Translated name']
    act_unit = compute_act_unit(df)
    
    len_df = len(df)
    
    act_data = [ ['Activity', act_name],
                 ['reference product',  act_name],
                 ['location', 'CH'],
                 ['amount', 1],
                 ['unit', act_unit] ]
    
    df_act = pd.DataFrame( act_data, 
                           columns=list('AB'),
                           index = np.arange(len_df,len_df+len(act_data)) )
                          
    df = df.append(df_act, sort=False)
    
    return df, df_act

In [ ]:
def append_exchanges_in_correct_columns(df, dict_with_values):
    '''
    Make sure that exchanges values are appended to df in the correct columns.
    '''  
    col_names = list(dict_with_values.keys()) # order of columns is determined by this list
    col_excel_literal = [EXC_COLUMNS_DICT[m] for m in col_names]
    
    if dict_with_values != EXC_COLUMNS_DICT:
        col_data  = [dict_with_values[m] for m in col_names]
    else:
        col_data = col_names
    
    df = df.append(pd.DataFrame([col_data], columns=col_excel_literal, index=[len(df)]), sort=False)
    
    return df

In [ ]:
def append_exchanges_column_names(df):
    '''
    Add column names for exchanges
    '''
    df = df.append(pd.DataFrame(['Exchanges'], columns=['A'], index=[len(df)]), sort=False)
    df = append_exchanges_in_correct_columns(df, EXC_COLUMNS_DICT)
    return df

In [ ]:
def append_first_exchange(df, df_act):
    '''
    Append first exchange which is activity itself, the amount is always 1, 
    the database is always the one that is being currently created, type is `production`.
    '''
    df_act_dict = df_act.set_index('A').to_dict()['B']
    
    first_exc_data_dict = { 'name': df_act_dict['Activity'],
                            'reference product': df_act_dict['reference product'],
                            'location': df_act_dict['location'],
                            'amount': 1,
                            'unit': df_act_dict['unit'],
                            'database': CONSUMPTION_DB_NAME,
                            'type': 'production',
                          }
    
    df = append_exchanges_in_correct_columns(df, first_exc_data_dict)
    
    return df

In [ ]:
def is_pattern_correct(df_ind_j):
    '''
    Check that input activity info has correct pattern. 
    In case the pattern is not correct, move on to the next 5 columns and check their pattern.
    This is needed because for some input activities some relevant values are missing, eg only 'On' value is present.
    '''
    list_ = list(df_ind_j.index)
    pattern = ['On', 'Activity', 'DB Act', 'CFL Act', 'Amount Act']
    check = [pattern[n] in list_[n] for n in range(N_COLUMNS_INPUT_ACTIVITY)]
    if np.all(check): 
        return 1
    else: 
        return 0

In [ ]:
def append_exchanges(df, df_ind, df_act):
    '''
    Add all exchanges (input activities) from the row df_ind to consumption database dataframe.
    '''
    # Add exchanges column names
    df = append_exchanges_column_names(df)
    
    # Add first exchange that is the same as the activity itself, type of this exchange is production
    df = append_first_exchange(df, df_act)
    
    # Add all exchanges
    n_exchanges = (len(df_ind)-FIRST_ACT_IND) // N_COLUMNS_INPUT_ACTIVITY
    if n_exchanges != (len(df_ind) - FIRST_ACT_IND) / N_COLUMNS_INPUT_ACTIVITY:
        print('smth is not right with exchanges of Activity -> ' + str(df_ind['Translated name']))
    
    ConversionDem2FU = df_ind['ConversionDem2FU']
    skip = 0
    for j in range(1, n_exchanges+1):
        
        start = FIRST_ACT_IND + N_COLUMNS_INPUT_ACTIVITY*(j-1) + skip
        end = start + N_COLUMNS_INPUT_ACTIVITY
        df_ind_j = df_ind[start:end]
        
        #Check that df_ind_j contains <On 1, Activity 1, DB Act 1, CFL Act 1, Amount Act 1> pattern
        flag = 1
        while flag:
            flag_pattern = is_pattern_correct(df_ind_j) 
            if flag_pattern == 1: # we don't need to skip if patter is correct
                flag = 0
            else:
                skip += 1
                start = FIRST_ACT_IND + N_COLUMNS_INPUT_ACTIVITY*(j-1) + skip
                end = start + N_COLUMNS_INPUT_ACTIVITY
                df_ind_j = df_ind[start:end]
        
        df = append_one_exchange(df, df_ind_j, ConversionDem2FU)
        
    return df

In [ ]:
def create_input_act_dict(act_bw, input_act_amount):
    '''
    Create a dictionary with all info about input activities.
    '''
    
    input_act_values_dict = {
        'name': act_bw['name'], 
        'location': act_bw['location'], 
        'amount': input_act_amount, 
        'unit': act_bw['unit'], 
        'database': act_bw['database'], 
        # We do not expect type biosphere, but assign it via ACTIVITY_TYPE_DICT anyway 
        # to be sure that we don't encounter them.
        'type': ACTIVITY_TYPE_DICT[act_bw['type']],
    }
    try:
        input_act_values_dict['reference product'] = act_bw['reference product']
    except:
        pass
            
    return input_act_values_dict

In [ ]:
def bw_get_activity_info_manually(input_act_str, db_name, input_act_amount):
    # Extract the activity name
    apostrophes = [(m.start(0), m.end(0)) for m in re.finditer("'", input_act_str)]
    if len(apostrophes) == 1:
        ap_start = 0
        ap_end = apostrophes[0][0]
    else:
        ap_start = apostrophes[0][1]
        ap_end = apostrophes[1][0]
    input_act_name = input_act_str[ ap_start:ap_end ]
    input_act_unit_loc = input_act_str[ input_act_str.find("(") : input_act_str.find(")")+1 ]
    input_act_unit_loc_split = [ re.sub('[^-A-Za-z0-9-€-]', ' ' , el).rstrip().lstrip() \
                                 for el in input_act_unit_loc.split(',')]
    input_act_unit = input_act_unit_loc_split[0]
    input_act_location = input_act_unit_loc_split[1]

    # Add comment when activity cannot be found
    input_act_values_dict = {}
    input_act_values_dict['name'] = input_act_name
    input_act_values_dict['unit'] = input_act_unit
    input_act_values_dict['location'] = input_act_location
    input_act_values_dict['amount'] = input_act_amount
    input_act_values_dict['database'] = db_name
    input_act_values_dict['type'] = ACTIVITY_TYPE_DICT['process'] # TODO remove hardcoding
    input_act_values_dict['comment'] = 'TODO could not find this activity'

    return input_act_values_dict

In [ ]:
 def append_one_exchange(df, df_ind_j, ConversionDem2FU):
    '''
    Extract information about one input activity, eg name, unit, location, etc and append it to the dataframe df.
    '''    
    # Extract the activity number
    k = int(''.join(c for c in df_ind_j.index[0] if c.isdigit()))
    # Extract information about activity and save it
    input_act_str = df_ind_j['DB Act ' + str(k)]
    input_act_db_code = df_ind_j['Activity ' + str(k)]
    
    # Find this input activity in brightway databases
    db_name = input_act_db_code.split("'")[1]
    code = input_act_db_code.split("'")[3]
    input_act_db_code_tuple = (db_name, code)
    
    # TODO remove HEIA for now
    if 'heia' in db_name:
        return df
    
    # Compute amount
    input_act_amount = ConversionDem2FU \
                     * df_ind_j['On ' + str(k)] \
                     * df_ind_j['CFL Act ' + str(k)] \
                     * df_ind_j['Amount Act ' + str(k)]
    
    try:
        # Find activity using bw functionality
        act_bw = bw.get_activity(input_act_db_code_tuple)
        input_act_values_dict = create_input_act_dict(act_bw, input_act_amount)
    except:
        # If bw.get_activity does not work for whichever reason, fill info manually
        input_act_values_dict = bw_get_activity_info_manually(input_act_str, db_name, input_act_amount)
        
    # Add exchange to the dataframe with database in brightway format
    df = append_exchanges_in_correct_columns(df, input_act_values_dict)
    
    return df

In [ ]:
# def append_one_exchange_old(df, df_ind_j, ConversionDem2FU):
#     '''
#     Extract information about one input activity, eg name, unit, location, etc and append it to the dataframe df.
#     '''    
#     # Extract the activity number
#     k = int(''.join(c for c in df_ind_j.index[0] if c.isdigit()))
    
#     input_act_str = df_ind['DB Act ' + str(k)]
    
#     # Extract the activity name
#     apostrophes = [(m.start(0), m.end(0)) for m in re.finditer("'", input_act_str)]
#     if len(apostrophes) == 1:
#         ap_start = 0
#         ap_end = apostrophes[0][0]
#     else:
#         ap_start = apostrophes[0][1]
#         ap_end = apostrophes[1][0]
#     input_act_name = input_act_str[ ap_start:ap_end ]
    
#     input_act_unit_loc = input_act_str[ input_act_str.find("(") : input_act_str.find(")")+1 ]
# #     input_act_unit_loc_split = [ re.sub('\W+', ' ' , el) for el in input_act_unit_loc.split(',')]
#     input_act_unit_loc_split = [ re.sub('[^-A-Za-z0-9-€-]', ' ' , el).rstrip().lstrip() \
#                                  for el in input_act_unit_loc.split(',')]
#     input_act_unit = input_act_unit_loc_split[0]
#     input_act_location = input_act_unit_loc_split[1]
    
#     # Extract input activity amount
#     input_act_amount = df_ind['On ' + str(k)] * df_ind['CFL Act ' + str(k)] * df_ind['Amount Act ' + str(k)]
    
#     # Find this input activity in brightway databases
#     input_act_db_code = df_ind['Activity ' + str(k)]
#     db_name = input_act_db_code.split("'")[1]
#     db = bw.Database(db_name)
    
#     # TODO remove HEIA for now
#     if 'heia' in db_name:
#         return df
    
#     if 'EXIOBASE' in db_name and input_act_location == 'CH':
#         input_act_location = 'Switzerland'
    
#     acts_bw = [act for act in db if  input_act_name == act['name'] \
#                                  and input_act_unit == act['unit'] \
#                                  and input_act_location == act['location']]
    
#     # TODO change this part once we get rid of non unique activities
#     try:
#         act_bw = acts_bw[0]   
#         input_act_values_dict = create_input_act_dict(act_bw, input_act_amount)
        
#         # Add comment when activity is not unique
#         if len(acts_bw) > 1:
#             input_act_values_dict['comment'] = 'TODO: not unique!'
            
#     except:
#         # Add comment when activity cannot be found
#         input_act_values_dict = {}
#         input_act_values_dict['name'] = input_act_name
#         input_act_values_dict['unit'] = input_act_unit
#         input_act_values_dict['location'] = input_act_location
#         input_act_values_dict['amount'] = input_act_amount
#         input_act_values_dict['database'] = db_name
#         input_act_values_dict['type'] = ACTIVITY_TYPE_DICT['process'] # TODO remove hardcoding
#         input_act_values_dict['comment'] = 'TODO: cannot find this activity'
        
#     # Add exchange to the dataframe with database in brightway format
#     df = append_exchanges_in_correct_columns(df, input_act_values_dict)
    
#     return df

# Convert data to brightway database format -> main code
calls all the functions used above

In [ ]:
%%time
# Start brightway project that already contains databases
project = 'GSA for ecoinvent'
bw.projects.set_current(project)

# Create dataframe that will be our consumption database after we add activities and exchanges from the raw file
df_bw = create_df_bw(CONSUMPTION_DB_NAME)

# Read data
path = 'data/es8b01452_si_002.xlsx'
sheet_name = 'Overview & LCA-Modeling'
df_raw = pd.read_excel(path, sheet_name = sheet_name, header=2)

# Add ON columns
df = complete_columns(df_raw)

act_indices = df_raw.index[df_raw['ConversionDem2FU'].notna()].tolist() # indices of all activities

for ind in act_indices:
    # For each row
    df_ind = df_raw.iloc[ind]
    df_ind = df_ind[df_ind.notna()]
    # Add activity
    df_bw, df_act = append_activity(df_bw, df_ind[:N_ACT_RELEVANT]) # only pass columns relevant to this function 
    # Add exchanges
    df_bw = append_exchanges(df_bw, df_ind, df_act)

In [ ]:
# Write the dataframe to excel file
write_dir_name = 'write_files'
if not os.path.exists(write_dir_name):
    os.mkdir(write_dir_name)
db_bw_path = write_dir_name + '/' + 'consumption_db.xlsx'
df_bw.to_excel(db_bw_path, index=False, header=False)

# ------------------------------------------ Part 2 ------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import brightway2 as bw
import string
from copy import copy, deepcopy

# Constants

In [2]:
DB_COLUMN = 'F'
CONSUMPTION_DB_NAME = 'CH consumption 1.0'

# Replace names of old databases with the new ones in the consumption database excel file

In [3]:
def replace_one_db(df, db_old_name, db_new_name):
    '''
    Replace database name with a new one (eg in case a newer version is available)
    '''
    df_updated = copy(df)
    
    where = np.where(df_updated[DB_COLUMN]==db_old_name)[0]
    if where.shape[0] != 0:
        df_updated[DB_COLUMN][where] = db_new_name
        
    return df_updated

In [4]:
def update_all_db(df):
    '''
    Update all databases in the consumption database
    '''
    db_old_list = ['Agribalyse 1.2', 
                   'ecoinvent 3.3 cutoff']
    db_new_list = ['Agribalyse 1.2 - ecoinvent 3.3 cutoff',
                  'ecoinvent 3.6 cutoff']
    
    assert len(db_old_list) == len(db_new_list)
    
    for i in range(len(db_old_list)):
        df = replace_one_db(df, db_old_list[i], db_new_list[i])
        
    return df

In [5]:
# Main code
project = 'GSA for ecoinvent'
bw.projects.set_current(project)

# Read consumption database
path = 'write_files/consumption_db.xlsx'
df = pd.read_excel(path, header = None)
df.columns = list(string.ascii_uppercase[:len(df.columns)])
# 
# Replace
df = update_all_db(df)
path_new_db = 'write_files/consumption_db_updated.xlsx'
df.to_excel(path_new_db, index=False, header=False)

# Import consumption database linked to older versions of other databases

# 1. Ecoinvent 3.6

### TODO Chris -> please check migrations

In [6]:
if CONSUMPTION_DB_NAME in bw.databases:
    print(CONSUMPTION_DB_NAME + " database already present!!! No import is needed")
else: 
    co = bw.ExcelImporter(path_new_db)
    co.apply_strategies()
    co.match_database('EXIOBASE 2.2', fields=('name','reference product', 'unit','location','categories'))
    co.match_database('ecoinvent 3.6 cutoff', fields=('name', 'reference product', 'unit','location','categories'))
    co.match_database('Agribalyse 1.2 - ecoinvent 3.3 cutoff', fields=('name','unit','location'))
    co.statistics()

Extracted 1 worksheets in 0.39 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.32 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
203 datasets
6058 exchanges
95 unlinked exch

In [7]:
# Define a migration for two particular activities that can only be hardcoded
ecoinvent36_change_names_data = {
    'fields': ['name', ],
    'data': [
        (
            ['steam production in chemical industry'], 
            {
                'name': 'steam production, in chemical industry',
                'reference product': 'steam, in chemical industry',
                'unit': 'kilogram',
                'multiplier': 1/2.75, # see comment on this activity in ecoinvent
            }
        ),
        (
            ['market for green bell pepper'],
            {
                'name': 'market for bell pepper',
                'reference product': 'bell pepper',
            }
        ),
    ]
}

bw.Migration("ecoinvent36-change-names").write(
    ecoinvent36_change_names_data,
    description="Change names of some activities"
)

In [8]:
# Define a migration for rice production and specific locations
# These locations have only non-basmati rice production
ecoinvent36_rice_production_data = {
    'fields': ['name', 'location'],
    'data': [
        (
            ['rice production', 'US'],
            {
                'name': 'rice production, non-basmati',
                'reference product': 'rice, non-basmati'
            }
        ),
        (
            ['rice production', 'CN'],
            {
                'name': 'rice production, non-basmati',
                'reference product': 'rice, non-basmati'
            }
        ),
    ]
}

bw.Migration("ecoinvent36-rice-production").write(
    ecoinvent36_rice_production_data,
    description="Change names of some activities"
)

In [9]:
co.migrate('ecoinvent36-change-names')
co.migrate("ecoinvent36-rice-production")
co.match_database('ecoinvent 3.6 cutoff', fields=('name','reference product', 'unit','location','categories'))
co.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
203 datasets
6058 exchanges
26 unlinked exchanges
  Type technosphere: 7 unique unlinked exchanges


(203, 6058, 26)

The rest of the unlinked exchanges are not uniquely defined in ecoinvent 3.6 -> 1-to-multiple mapping. <br>
For example 'rice production' is now divided into basmati and non-basmati rice. <br>
Hence, we split them based on their shares in the production volumes.

In [10]:
# Manually choose which ecoinvent 3.6 exchanges should be taken for each unlinked exchange
ei36 = bw.Database('ecoinvent 3.6 cutoff')
mapping = [
    {('market for rice', 'GLO'): 
        [act['code'] for act in ei36 if  'market for rice' in act['name'] 
                                     and act['location']=='GLO'
                                     and 'seed' not in act['name']]},
    
    {('rice production', 'RoW'): 
        [act['code'] for act in ei36 if  'rice production' in act['name'] 
                                     and act['location']=='RoW'
                                     and 'straw' not in act['reference product']]},
    
    {('rice production', 'IN'): 
        [act['code'] for act in ei36 if  'rice production' in act['name'] 
                                     and act['location']=='IN'
                                     and 'straw' not in act['reference product']]},
    
    {('market for wheat grain', 'GLO'): 
        [act['code'] for act in ei36 if  'market for wheat grain' in act['name'] 
                                     and 'feed' not in act['name']]},
    
    {('market for maize grain', 'GLO'): 
        [act['code'] for act in ei36 if  'market for maize grain' in act['name'] 
                                     and 'feed' not in act['name']]},
    
    {('market for mandarin', 'GLO'): 
        [act['code'] for act in ei36 if 'market for mandarin' in act['name']]},
    
    {('market for soybean', 'GLO'): 
        [act['code'] for act in ei36 if 'market for soybean' in act['name'] 
                             and all([_ not in act['name'] for _ in ['meal','beverage','seed','feed','oil']] )]},
]

In [11]:
def get_allocated_excs(mapping):
    '''
    Function that has the same length as mapping, returns list of lists.
    Each "inner" list contains dictionaries of exchanges in the correct format. 
    By correct format we mean the one consistent with the format of exchanges 
    in the (not written yet) consumption database (see eg co.data[0]['exchanges']).
    We exclude amounts for now and instead add field 'production volume share'
    '''
    
    ei36_name = 'ecoinvent 3.6 cutoff'
    allocated_excs = [[]]*len(mapping)
    
    for m in range(len(mapping)):
        new_exchanges = []
        vols = 0
        codes = list(mapping[m].values())[0]
        for code in codes:
            act = bw.get_activity((ei36_name, code))
            production_exc = next(item for item in act.exchanges() if item['type']=='production')
            vol = production_exc['production volume']
            vols += vol

            exc = {'name': act['name'],
                   'reference product': act['reference product'],
                   'location': act['location'],
                   'production volume share': vol,
                   'unit': act['unit'],
                   'database': ei36_name,
                   'type': 'technosphere'}

            new_exchanges.append(exc)
            
        for exc in new_exchanges:
            exc['production volume share'] /= vols
            
        allocated_excs[m] = new_exchanges
        
    return allocated_excs

In [46]:
def modify_exchanges(co, mapping):
    '''
    Function that finds all activities that have exchanges that should be split into multiple other exchanges 
    in a newer ecoinvent version based on production volume allocation principle, removes old exchanges
    and adds new ones with correct amounts.
    '''
    
    co1 = deepcopy(co)
    unlinked_list = list(co1.unlinked)
    allocated_excs = get_allocated_excs(mapping)
        
    for act in co1.data:        
        exchanges = deepcopy(act['exchanges'])
        new_exchanges = []
        ind_amount = []
        
        for exc in exchanges:
            ind = next((i for i, item in enumerate(unlinked_list) if all([exc[f]==item[f] 
                          for f in ['name','location','unit','database','type']])), None)
            
            if ind != None:
                # if we find current exchange in the unlinked exchanges list, replace it with other ones
                # while using allocation by production volume
                allocated_excs_new_amt = deepcopy(allocated_excs[ind])
                for exc_new_amt in allocated_excs_new_amt:
                    exc_new_amt['amount'] = exc_new_amt['production volume share'] * exc['amount']
                [new_exchanges.append(all_exc) for all_exc in allocated_excs_new_amt]
            else:
                # if we don't find current exchange in the unlinked exchanges list, append current to the list
                new_exchanges.append(exc)

        act['exchanges'] = new_exchanges 

    co1.match_database('ecoinvent 3.6 cutoff', fields=('name','reference product', 'unit','location'))
    
    return co1

In [49]:
co = modify_exchanges(co, mapping)
co.statistics()

Applying strategy: link_iterable_by_fields
203 datasets
6119 exchanges
0 unlinked exchanges
  


(203, 6119, 0)

# Playground